In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ Wczytanie modelu
model = load_model("lstm_weather_best_all_features.h5", compile=False)

# 2️⃣ Wczytanie danych historycznych
df = pd.read_csv("european_capitals_history_clean.csv")

# 3️⃣ Wybór miasta
city_name = "Warszawa"
city_data = df[df['city'] == city_name].sort_values('date')

# 4️⃣ Przygotowanie wszystkich cech dla ostatnich 14 dni
features = ['tavg','tmin','tmax','prcp','wspd','pres']  # przykład
X_input = city_data[features].values[-14:]  # ostatnie 14 dni

# 5️⃣ Normalizacja wejścia tym samym scalerem co trening
scaler = MinMaxScaler(feature_range=(0,1))
X_scaled = scaler.fit_transform(X_input)  # jeśli masz zapisany scaler, użyj load
X_scaled = X_scaled.reshape(1, 14, len(features))

# 6️⃣ Predykcja
y_pred_scaled = model.predict(X_scaled)  # shape: (1, 7, 1)

# 7️⃣ Odeskowanie wyników
scaler_y = MinMaxScaler(feature_range=(0,1))
scaler_y.fit(city_data[['tavg']])  # jeśli masz zapisany scaler_y, użyj load
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1,1)).flatten()

# 8️⃣ Tworzenie DataFrame z prognozą
future_dates = pd.date_range(
    start=pd.to_datetime(city_data['date'].iloc[-1]) + pd.Timedelta(days=1),
    periods=7,
    freq='D'
)

df_forecast = pd.DataFrame({
    'date': future_dates,
    'tavg_pred': y_pred,
    'city': city_name
})

print("📅 Prognoza 7-dniowa średniej temperatury dla miasta:", city_name)
print(df_forecast)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
📅 Prognoza 7-dniowa średniej temperatury dla miasta: Warszawa
        date  tavg_pred      city
0 2025-11-06   5.204278  Warszawa
1 2025-11-07   2.846644  Warszawa
2 2025-11-08   2.364425  Warszawa
3 2025-11-09   1.761329  Warszawa
4 2025-11-10   1.331974  Warszawa
5 2025-11-11   1.219558  Warszawa
6 2025-11-12   0.586895  Warszawa
